## Deliverable 2. Create a Customer Travel Destinations Map.

In [53]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [54]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kautokeino,NO,69.0125,23.0412,5.22,96,100,3.85,overcast clouds
1,1,Ancud,CL,-41.8697,-73.8203,58.53,58,100,9.57,overcast clouds
2,2,Sinnamary,GF,5.3833,-52.9500,80.26,85,92,7.85,overcast clouds
3,3,Mildura,AU,-34.2000,142.1500,66.16,59,75,9.22,broken clouds
4,4,Avarua,CK,-21.2078,-159.7750,77.05,94,90,3.44,light rain


In [40]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=int(input("Enter minimum temperature:"))
max_temp=int(input("Enter maximum temperature:"))

Enter minimum temperature:0
Enter maximum temperature:100


In [43]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df["Max Temp"]
temp_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
temp_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kautokeino,NO,69.0125,23.0412,5.22,96,100,3.85,overcast clouds
1,1,Ancud,CL,-41.8697,-73.8203,58.53,58,100,9.57,overcast clouds
2,2,Sinnamary,GF,5.3833,-52.9500,80.26,85,92,7.85,overcast clouds
3,3,Mildura,AU,-34.2000,142.1500,66.16,59,75,9.22,broken clouds
4,4,Avarua,CK,-21.2078,-159.7750,77.05,94,90,3.44,light rain
...,...,...,...,...,...,...,...,...,...,...
722,722,Ginir,ET,7.1333,40.7000,62.98,51,100,3.44,overcast clouds
723,723,Galle,LK,6.0367,80.2170,79.56,84,100,3.56,overcast clouds
724,724,Lengshuijiang,CN,27.6881,111.4294,51.67,71,0,2.84,clear sky
725,725,Ambon,ID,-3.6954,128.1814,81.81,78,96,4.59,overcast clouds


In [44]:
# 4a. Determine if there are any empty rows.
temp_df.isnull()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
722,False,False,False,False,False,False,False,False,False,False
723,False,False,False,False,False,False,False,False,False,False
724,False,False,False,False,False,False,False,False,False,False
725,False,False,False,False,False,False,False,False,False,False


In [56]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=temp_df.dropna()

In [70]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
#stick to US and Canada
hotel_df=hotel_df.loc[(hotel_df["Country"]=="US") | (hotel_df["Country"]=="CA")]

In [71]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"#default coords
}

# 6d. Set up the base URL for the Google Directions API to get JSON data.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

print("Beginning Data Retrieval     ")
print("-----------------------------")

# 6b. Iterate through the hotel DataFrame.
for i,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat}, {lng}"
    name=row["City"]
    
    # 6e. Make request and retrieve the JSON data from the search. 
    g_json=requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    if len(g_json["results"]) == 0:
        print(f"Hotel in {name} not found. Skipping...")
    else:
#         print(f"Hotel in {name} found. ")
        hotel_df.loc[i, "Hotel Name"] =g_json["results"][0]["name"]
    

print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Hotel in Albany not found. Skipping...
Hotel in Avera not found. Skipping...
Hotel in Saint-Augustin not found. Skipping...
-----------------------------
Data Retrieval Complete      
-----------------------------


In [72]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Torbay,CA,47.17,overcast clouds,47.6666,-52.7314,Gallows Cove
11,Kapaa,US,75.52,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
20,Aklavik,CA,2.23,light snow,68.2191,-135.0107,Aklavik Inn
21,Yellowknife,CA,12.72,snow,62.4560,-114.3525,The Explorer Hotel
26,Jamestown,US,33.12,snow,42.0970,-79.2353,Hampton Inn & Suites Jamestown
...,...,...,...,...,...,...,...
668,Moose Factory,CA,4.98,overcast clouds,51.2585,-80.6094,Moosonee Lilypad
673,Greenville,US,59.02,scattered clouds,35.6127,-77.3663,the 5th Street Inn
677,Kitimat,CA,34.18,light rain,54.0524,-128.6534,Kingfish Westcoast Adventures Ltd. Kitimat Est...
703,Smithers,CA,30.52,scattered clouds,54.7804,-127.1743,"Prestige Hudson Bay Lodge, Premier Collection"


In [73]:
# 8a. Create the output File (CSV)
output_data_file="Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [74]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [75]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#              max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))